## GeoCat experimental

In [2]:
import importlib
from collections import defaultdict
import numpy as np
import pandas as pd
import lib.recommenders as rec
import networkx as nx
import matplotlib.pyplot as plt
import lib.areamanager
import math
import time
from IPython.core.debugger import set_trace
import lib.geocat.objfunc as gcobjfunc
import lib.cat_utils as cat_utils

from lib.constants import experiment_constants
import lib.metrics as metrics
import pickle
%matplotlib inline

from lib.metricscontroller import MetricsController

import lib.pgc.pgc as pgc

def string_to_array(string):
    if string == '':
        return list()
    return eval(string)
importlib.reload(lib.metricscontroller)
importlib.reload(lib.constants)
importlib.reload(rec)
importlib.reload(gcobjfunc)
importlib.reload(cat_utils)
importlib.reload(lib.metricscontroller)
N=experiment_constants.N
K=experiment_constants.K
CITY=experiment_constants.CITY
METRICS_K=experiment_constants.METRICS_K



In [3]:
#Train load
data_checkin_train = pickle.load(open("../data/checkin/train/"+CITY+".pickle","rb"))

#Test load
ground_truth = defaultdict(set)
for checkin in pickle.load(open("../data/checkin/test/"+CITY+".pickle","rb")):
    ground_truth[checkin['user_id']].add(checkin['poi_id'])
#Pois load
poi_coos = {}
poi_cats = {}
for poi_id,poi in pickle.load(open("../data/poi/"+CITY+".pickle","rb")).items():
    poi_coos[poi_id] = tuple([poi['latitude'],poi['longitude']])
    poi_cats[poi_id] = poi['categories']
    
#Social relations load
social_relations = defaultdict(list)
for user_id,friends in pickle.load(open("../data/user/"+CITY+".pickle","rb")).items():
    social_relations[user_id]=friends

user_num = len(social_relations)
poi_num = len(poi_coos)
user_num,poi_num

# Cat Hierarchy load
dict_alias_title,category_tree,dict_alias_depth=cat_utils.cat_structs("../data/categories.json")
undirected_category_tree=category_tree.to_undirected()

# Training matrix create
training_matrix = np.zeros((user_num, poi_num))
for checkin in data_checkin_train:
    training_matrix[checkin['user_id'],checkin['poi_id']]+=1

# poi neighbors load
poi_neighbors=pickle.load(open("../data/neighbor/"+CITY+".pickle","rb"))

In [ ]:
# df_checkin=pd.read_csv("../data/checkin/"+CITY+".csv",converters={'categories':string_to_array})

# df_checkin_train=pd.read_csv("../data/checkin/train/"+CITY+".csv",converters={'categories':string_to_array})


# users_id=df_checkin['user_id'].drop_duplicates().reset_index(drop=True)
# pois_id=df_checkin['business_id'].drop_duplicates().reset_index(drop=True)
# user_num=len(users_id)
# poi_num=len(pois_id)


# df_checkin_train=df_checkin_train.set_index('user_id')

# df_checkin_train.head()

In [ ]:
# len(df_checkin_test)

In [ ]:
# len(df_checkin_train)

In [ ]:
# df_checkin_test=pd.read_csv("../data/checkin/test/"+CITY+".csv",converters={'categories':string_to_array})

In [ ]:
# ground_truth = defaultdict(set)
# for index,row in df_checkin_test[['user_id','business_id']].drop_duplicates().iterrows():
#     ground_truth[row['user_id']].add(row['business_id'])

In [ ]:
# poi_coos = {}
# for lid,latlon in df_checkin[['business_id','latitude','longitude']].reset_index(drop=True).drop_duplicates().set_index('business_id').iterrows():
#     poi_coos[lid] = tuple(latlon)

In [ ]:
pgc.cmean_dist_pois(poi_coos)

In [ ]:
# poi_cats = {}
# for lid,row in df_checkin[['business_id','categories']].reset_index(drop=True).drop_duplicates(subset=['business_id']).set_index('business_id').iterrows():
#     poi_cats[lid] = row.categories

In [ ]:
# dict_alias_title,category_tree,dict_alias_depth=cat_utils.cat_structs()
# undirected_category_tree=category_tree.to_undirected()

In [ ]:
# training_matrix = np.zeros((user_num, poi_num))
# for user_id,poi_id in df_checkin_train.business_id.iteritems():
#    # print(user_id,poi_id)
#     training_matrix[user_id,poi_id]+=1

In [ ]:
all_uids = list(range(user_num))
all_lids = list(range(poi_num))

#np.random.shuffle(all_uids)

In [ ]:
user_predict={}
for index,row in pd.read_csv("../data/result/reclist/sigir11_top_100.txt",header=None).iterrows():
    user_id=row[0]
    lid_list=row[1]
    user_predict[user_id]=eval(lid_list)

In [ ]:
# df_poi_neighbor=pd.read_csv("../data/neighbor/"+CITY+".csv",converters={'neighbors':string_to_array})
# df_poi_neighbor=df_poi_neighbor.set_index("business_id")
# poi_neighbors={}
# for lid,row in df_poi_neighbor.iterrows():
#     neighbors=row['neighbors']
#     poi_neighbors[lid]=neighbors
# df_poi_neighbor=None

In [ ]:


#print(MetricsController(metrics=['precision','recall','pr','ild','gc'],algorithm='usg',k=5))

In [ ]:
# for i in mcs:
#     print(i)

In [ ]:
importlib.reload(rec)
importlib.reload(gcobjfunc)
#precision,recall,ild,gc,pr = [],[],[],[],[]

range_K=range(K)
overall_scores=np.zeros(poi_num)
mcs=list()
for k in METRICS_K:
    mcs.append(MetricsController(metrics=['precision','recall','pr','ild','gc','epc'],algorithm='mostpopular_geocat',k=k))
pop=epc_pop_list(training_matrix)
epc_numerator=list()
epc_denominator=list()

for cnt, uid in enumerate(all_uids):
    if uid in ground_truth:
        
        pois_score=rec.mostpopularnp(training_matrix,uid)
        tmp_rec_list=list(reversed(np.argsort(pois_score)))[0:N]
        tmp_score_list=list(reversed(np.sort(pois_score)))[0:N]
        
        actual=ground_truth[uid]
        predicted=tmp_rec_list
        
        #print(list(reversed(np.sort(pois_score)))[0:N])
        
        #print(tmp_rec_list)
        #break
        #tmp_rec_list=user_predict[uid][:N]
        rec_list=[]
        
        lids=training_matrix[uid].nonzero()[0]
        
        lid_visits=training_matrix[:,lids].sum(axis=0)
        mean_visits=lid_visits.mean()
        relevant_lids=lids[lid_visits>mean_visits]
        relevant_cats=set()
        for lid in relevant_lids:
            relevant_cats.update(poi_cats[lid])
        # log_size=training_matrix[0,training_matrix[0,:].nonzero()[0]].sum()
        user_log=training_matrix[uid]
        #print(mean_visits)
        log_poi_ids=list()
        poi_cover=list()
        for lid in user_log.nonzero()[0]:
            for visits in range(int(user_log[lid])):
                poi_cover.append(0)
                log_poi_ids.append(lid)
        log_size=len(log_poi_ids)
        assert user_log[user_log.nonzero()[0]].sum() == len(poi_cover)
#         print(uid)
#         print("Count:",cnt)
        div_geo_cat_weight = 0.75 # beta,this is here because of the work to be done on parameter customization for each user
        div_weight = 0.5 # lambda, geo vs cat
        current_proportionality=0
        final_scores=[]
        
        start_time = time.time()
        log_neighbors=dict()
        for poi_id in tmp_rec_list:
            neighbors=list()
            for id_neighbor in poi_neighbors[poi_id]:
                for i in range(log_size):
                    log_poi_id=log_poi_ids[i]
                    if log_poi_id == id_neighbor:
                        neighbors.append(i)
            log_neighbors[poi_id]=neighbors

        
        for i in range_K:
            #print(i)
            poi_to_insert=None
            max_objective_value=-200
            for j in range(len(tmp_rec_list)):
                candidate_poi_id=tmp_rec_list[j]
                candidate_score=tmp_score_list[j]
                ild_div=gcobjfunc.min_dist_to_list_cat(candidate_poi_id,rec_list,poi_cats,undirected_category_tree)
                gc_div=gcobjfunc.gc(candidate_poi_id,rec_list,relevant_cats,poi_cats)
                pr=gcobjfunc.update_geo_cov(candidate_poi_id,log_poi_ids,K,poi_cover.copy(),poi_neighbors,log_neighbors[candidate_poi_id])
                
                objective_value=gcobjfunc.ILD_GC_PR(candidate_score,ild_div,gc_div,pr,current_proportionality,K,div_geo_cat_weight,div_weight)
                #print(candidate_poi_id,ild_div,gc_div,max(0,pr-current_proportionality),objective_value)
                #print(candidate_poi_id,objective_value)

                if objective_value > max_objective_value:
                    max_objective_value=objective_value
                    poi_to_insert=candidate_poi_id
                pass
            if poi_to_insert is not None:
                #print(poi_to_insert,max_objective_value)
                
                rm_idx=tmp_rec_list.index(poi_to_insert)
                
                tmp_rec_list.pop(rm_idx)
                tmp_score_list.pop(rm_idx)
                rec_list.append(poi_to_insert)
                final_scores.append(max_objective_value)
                # remove from tmp_rec_list
                current_proportionality=gcobjfunc.update_geo_cov(poi_to_insert,log_poi_ids,K,poi_cover,poi_neighbors,log_neighbors[poi_to_insert])
                #print(current_proportionality)
        print("time → %fs" % (time.time()-start_time))
#         predicted=np.array(rec_list)[list(reversed(np.argsort(final_scores)))]
        
        for i,k in enumerate(METRICS_K):
            predicted_at_k=predicted[:k]
            
            precision_val=metrics.precisionk(actual, predicted_at_k)
            rec_val=metrics.recallk(actual, predicted_at_k)
            pr_val=metrics.prk(training_matrix[uid],predicted_at_k,poi_neighbors)
            ild_val=metrics.ildk(predicted_at_k,poi_cats,undirected_category_tree)
            gc_val=metrics.gck(uid,training_matrix,poi_cats,predicted_at_k)
            
            epc_val=metrics.epck(predicted_at_k,actual,uid,pop,epc_numerator,epc_denominator)
            
            mcs[i].append_data([precision_val,rec_val,pr_val,ild_val,gc_val,epc_val])
            print("userid →",uid,end=',')
            mcs[i].print_metrics()
            
            
            
#,f"prec@{N}: {np.mean(precision)}, rec@{N}: {np.mean(recall)}")
#         print(final_scores)
#         print(rec_list)
#         print(list(reversed(np.sort(final_scores))))
#         print(np.array(rec_list)[list(reversed(np.argsort(final_scores)))])
#         print()
for i in mcs:
    i.fwrite_metrics()